# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_file_location = "Resources/cit_df.csv"

city_df = pd.read_csv(city_file_location)

city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,0,Cidreira,0.0,BR,2020-01-26 14:56:07,62.0,-30.18,-50.21,78.21,7.85
1,1,Isangel,82.0,VU,2020-01-26 14:56:08,81.0,-19.55,169.27,78.22,13.80
2,2,Vaini,0.0,TO,2020-01-26 14:56:09,86.0,-21.20,-175.20,79.07,12.08
3,3,Linqu,100.0,CN,2020-01-26 14:56:10,72.0,36.52,118.54,35.01,1.01
4,4,Goderich,100.0,CA,2020-01-26 14:54:56,94.0,43.75,-81.72,36.00,11.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Access maps with unique API key
gmaps.configure(g_key)


In [4]:
#Set parameters for gmap
locations = city_df[["Lat", "Lon"]]
humidity = city_df["Humidity"].astype(float)

#Plot the heatmap
figure = gmaps.figure(map_type='HYBRID')

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=50,point_radius=1.5)

#add the layer to the figure
figure.add_layer(heat_layer)

#display figure
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#create a new df to allow for weather criteria. Max Temp lower than 80 bu thigher than 70
#wind speed less than 10 mph, zero cloudiness

conditions_df = city_df.copy()
conditions_df = conditions_df[conditions_df['Max Temp'] >=70]
conditions_df = conditions_df[conditions_df['Max Temp'] <=80]
conditions_df = conditions_df[conditions_df['Wind Speed'] <=10]
conditions_df = conditions_df[conditions_df['Cloudiness'] ==0]
conditions_df = conditions_df.dropna()
conditions_df = conditions_df.reset_index()

conditions_df_drop = conditions_df.drop(['index', 'Unnamed: 0'], axis=1)
conditions_df_drop

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,Cidreira,0.0,BR,2020-01-26 14:56:07,62.0,-30.18,-50.21,78.21,7.85
1,Kahului,0.0,US,2020-01-26 14:56:42,77.0,20.89,-156.47,70.00,5.01
2,Hermanus,0.0,ZA,2020-01-26 14:56:44,57.0,-34.42,19.23,71.01,8.99
3,Lebu,0.0,CL,2020-01-26 14:56:48,52.0,-37.62,-73.65,74.71,9.01
4,Arraial do Cabo,0.0,BR,2020-01-26 14:58:27,75.0,-22.97,-42.03,75.20,6.04
5,San Francisco del Mar Viejo,0.0,MX,2020-01-26 14:59:33,77.0,16.23,-94.65,76.96,4.00
6,Mecca,0.0,SA,2020-01-26 15:00:09,12.0,21.43,39.83,73.40,3.94
7,Constitucion,0.0,CL,2020-01-26 15:00:47,56.0,-35.33,-72.42,74.25,9.46
8,Ciudad Lazaro Cardenas,0.0,MX,2020-01-26 15:00:01,73.0,17.96,-102.20,74.26,2.59
9,Chicomuselo,0.0,MX,2020-01-26 15:04:03,65.0,15.77,-92.27,73.56,3.22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(conditions_df_drop, columns=["City", "Country", "Lat", "Lon"])

hotel_df["Hotel Name"]= ""

hotel_df


,City,Country,Lat,Lon,Hotel Name
0,Cidreira,BR,-30.18,-50.21,
1,Kahului,US,20.89,-156.47,
2,Hermanus,ZA,-34.42,19.23,
3,Lebu,CL,-37.62,-73.65,
4,Arraial do Cabo,BR,-22.97,-42.03,
5,San Francisco del Mar Viejo,MX,16.23,-94.65,
6,Mecca,SA,21.43,39.83,
7,Constitucion,CL,-35.33,-72.42,
8,Ciudad Lazaro Cardenas,MX,17.96,-102.20,
9,Chicomuselo,MX,15.77,-92.27,


In [7]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)
        
#apend dataframe with hotel names        
hotel_df['Hotel Name'] = hotel_name
hotel_df

,City,Country,Lat,Lon,Hotel Name
0,Cidreira,BR,-30.18,-50.21,Pousada Rosa
1,Kahului,US,20.89,-156.47,Courtyard by Marriott Maui Kahului Airport
2,Hermanus,ZA,-34.42,19.23,Birkenhead House
3,Lebu,CL,-37.62,-73.65,Hotel Plaza Lebu
4,Arraial do Cabo,BR,-22.97,-42.03,Hotel da Canoa
5,San Francisco del Mar Viejo,MX,16.23,-94.65,NaN
6,Mecca,SA,21.43,39.83,Hotel Fairmont Makkah Clock Royal Tower
7,Constitucion,CL,-35.33,-72.42,Hotel Montt Plaza
8,Ciudad Lazaro Cardenas,MX,17.96,-102.20,Hotel de Casablanca
9,Chicomuselo,MX,15.77,-92.27,Hotel San Carlos


In [8]:
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Lat,Lon,Hotel Name
0,Cidreira,BR,-30.18,-50.21,Pousada Rosa
1,Kahului,US,20.89,-156.47,Courtyard by Marriott Maui Kahului Airport
2,Hermanus,ZA,-34.42,19.23,Birkenhead House
3,Lebu,CL,-37.62,-73.65,Hotel Plaza Lebu
4,Arraial do Cabo,BR,-22.97,-42.03,Hotel da Canoa
6,Mecca,SA,21.43,39.83,Hotel Fairmont Makkah Clock Royal Tower
7,Constitucion,CL,-35.33,-72.42,Hotel Montt Plaza
8,Ciudad Lazaro Cardenas,MX,17.96,-102.20,Hotel de Casablanca
9,Chicomuselo,MX,15.77,-92.27,Hotel San Carlos
10,Mildura,AU,-34.20,142.15,Best Western Chaffey International Motor Inn


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = clean_hotel_df[["Lat", "Lon"]]

In [11]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))